In [ ]:
!pip3 install shapely geopandas geo

In [7]:
# Before this you need to have a MySQL database with 3wifi data

db_ip="localhost"
db_user="root"
db_pass="new_password"
db_name="p3wifi"

### Target: get Country Wifi map for specific country from public sources

#### Database connection

In [5]:
import mysql.connector

# Step 1: Retrieve data from MySQL database 
conn = mysql.connector.connect(
    host=f'{db_ip}',
    user=f'{db_user}',
    password=f'{db_pass}',
    database=f'{db_name}',
    charset='utf8',
    collation='utf8mb4_general_ci'  # Set a compatible collation
)
if conn.is_connected():
    print("Connected to MySQL database")

# Define country rectangle (Czechia for example)
latitude_min = 48.474378
latitude_max = 51.045193
longitude_min = 12.068481
longitude_max = 18.860779

country_geojson = '../../data/raw/geoBoundaries-CZE-ADM0.geojson'
raw_list = '../../data/clean/Wifi_passlist.txt'

Connected to MySQL database


#### SQL database -> CSV

In [8]:
# data saved to rectangle aproimation to ../inputs/COUNTRY_RECT.csv with
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(
    f'mysql+mysqlconnector://{db_user}:{db_pass}@{db_ip}/{db_name}',
    connect_args={'charset': 'utf8', 'collation': 'utf8mb4_general_ci'}
)

sql_query = f"""
SELECT nets.BSSID, ESSID, WifiKey, latitude, longitude
FROM nets
JOIN geo ON nets.BSSID = geo.BSSID
WHERE latitude BETWEEN {latitude_min} AND {latitude_max}
AND longitude BETWEEN {longitude_min} AND {longitude_max}
"""
df = pd.read_sql_query(sql_query, engine)

df.to_csv('../../data/clean/COUNTRY_RECT.csv', index=False)

In [9]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Read the GeoJSON file
gdf = gpd.read_file(f"{country_geojson}")

# Extract the polygon
polygon = gdf.geometry.iloc[0]

# Read the CSV data
csv_data = pd.read_csv('../../data/clean/COUNTRY_RECT.csv')
def in_polygon(lon, lat):
    point = Point(lon, lat)
    return polygon.contains(point)

#Get Wifi passlist, not unique, not sorted
with open(f"{raw_list}", 'w') as outfile:
    for WifiKey, lon, lat in zip(csv_data['WifiKey'], csv_data['longitude'], csv_data['latitude']):
        if WifiKey not in {"<empty>", "nan","<not accessible>"} and in_polygon(lon, lat):
            print(WifiKey, file=outfile)

In [ ]:
!pwd
!head ./../../data/clean/Wifi_passlist.txt

In [18]:
!sort ./../../data/clean/Wifi_passlist.txt | uniq -c | sort -nr | awk '{print $2}' > ../../data/clean/Wifi_passlist_sorted_.txt

In [ ]:
#sorted passwords by count
!sort ./../../data/clean/Wifi_passlist.txt | uniq -c | sort -nr > ../../data/clean/Wifi_passlist_sorted.txt